<a href="https://colab.research.google.com/github/mistryvivek/flowers-nn-int2/blob/main/Model2SimpleWithBiggerChannels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms

In [2]:
# Download training data from open datasets.
training_data = datasets.Flowers102(
    root="data",
    split="train",
    download=True,
    transform = transforms.Compose([transforms.Resize((32, 32)), transforms.ToTensor()]),
)

# Download test data from open datasets.
test_data = datasets.Flowers102(
    root="data",
    split="test",
    download=True,
    transform = transforms.Compose([transforms.Resize((32, 32)), transforms.ToTensor()]),
)

100%|██████████| 344862509/344862509 [00:09<00:00, 35931758.56it/s]


Extracting data/flowers-102/102flowers.tgz to data/flowers-102


100%|██████████| 502/502 [00:00<00:00, 393779.80it/s]


100%|██████████| 14989/14989 [00:00<00:00, 10352119.65it/s]


In [23]:
batch_size = 30
num_classes = 102
learning_rate = 0.001
num_epochs = 100

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [24]:
import torch.nn.functional as F

class Net(nn.Module):
	#  Determine what layers and their order in CNN object 
    def __init__(self, num_classes):
        super(Net, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding_mode='replicate')
        self.conv_layer2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding_mode='replicate')
        self.max_pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.conv_layer3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding_mode='replicate')
        self.conv_layer4 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding_mode='replicate')
        self.max_pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.fc1 = nn.Linear(12800 , 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)
    

    def forward(self, x):
        out = self.conv_layer1(x)
        out = self.conv_layer2(out)
        out = self.max_pool1(out)
        
        out = self.conv_layer3(out)
        out = self.conv_layer4(out)
        out = self.max_pool2(out)
                
        out = out.reshape(out.size(0), -1)
        
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out


net = Net(num_classes)

In [25]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [26]:
# We use the pre-defined number of epochs to determine how many iterations to train the network on
for epoch in range(num_epochs):
	#Load in the data in batches using the train_loader object
    for i, (images, labels) in enumerate(train_dataloader):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = net(images)
        loss = criterion(outputs, labels)
      
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Epoch [1/100], Loss: 4.6306
Epoch [2/100], Loss: 4.6245
Epoch [3/100], Loss: 4.6262
Epoch [4/100], Loss: 4.6026
Epoch [5/100], Loss: 4.6056
Epoch [6/100], Loss: 4.6108
Epoch [7/100], Loss: 4.6109
Epoch [8/100], Loss: 4.5922
Epoch [9/100], Loss: 4.5767
Epoch [10/100], Loss: 4.5852
Epoch [11/100], Loss: 4.5626
Epoch [12/100], Loss: 4.5452
Epoch [13/100], Loss: 4.5020
Epoch [14/100], Loss: 4.5088
Epoch [15/100], Loss: 4.3890
Epoch [16/100], Loss: 4.3830
Epoch [17/100], Loss: 4.3425
Epoch [18/100], Loss: 4.1407
Epoch [19/100], Loss: 4.2843
Epoch [20/100], Loss: 4.1915
Epoch [21/100], Loss: 3.9458
Epoch [22/100], Loss: 3.6800
Epoch [23/100], Loss: 3.9694
Epoch [24/100], Loss: 3.8729
Epoch [25/100], Loss: 3.8797
Epoch [26/100], Loss: 3.6143
Epoch [27/100], Loss: 3.6504
Epoch [28/100], Loss: 3.7584
Epoch [29/100], Loss: 2.9675
Epoch [30/100], Loss: 3.0117
Epoch [31/100], Loss: 3.4580
Epoch [32/100], Loss: 3.0779
Epoch [33/100], Loss: 2.9418
Epoch [34/100], Loss: 2.5629
Epoch [35/100], Loss: 3

In [28]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_dataloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Accuracy of the network on the {} train images: {} %'.format(total, 100 * correct / total))

Accuracy of the network on the 6149 train images: 18.116766953976256 %
